If I use Tequila:

In [2]:
import tequila as tq
from tequila.quantumchemistry import optimize_orbitals
te_circuit = tq.import_open_qasm_from_file(filename='./H2.qasm')
mol = tq.Molecule(geometry="H 0.0 0.0 0.0\nH 0.0 0.0 1.5", basis_set="sto-3g")
H = mol.make_hamiltonian()
#wfn = tq.simulate(te_circuit, backend='qiskit')
E = tq.ExpectationValue(H=H, U=te_circuit)
result = tq.simulate(E)
result

converged SCF energy = -0.910873554594386


-0.890584781399921

In [7]:
from qiskit.qasm2 import load
from qiskit.primitives import Estimator
from qiskit_nature import settings
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
settings.use_pauli_sum_op = False

dist = 1.5# A
molecule = MoleculeInfo(
    #["H", "H","H","H"], [(0.0, 0.0, 0.0), (0.0, 0.0, dist), (0.0, 0.0,dist*2), (0.0, 0.0, dist*3)],
    ["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, dist),],
    #['Be','H','H'],[(0.0,0.0,0.0),(0.0,0.0,dist),(0.0,0.0,-dist)],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)
driver = PySCFDriver().from_molecule(molecule)
problem = driver.run()

problem_qcschema = driver.to_qcschema() #OO pyscf
mapper = JordanWignerMapper() 
hamiltonian = mapper.map(problem.second_q_ops()[0])

circuit = load(filename='./H2.qasm')
estimator = Estimator()
job = estimator.run(circuits=circuit,observables=hamiltonian)
result = job.result()
result.values[0] + problem.nuclear_repulsion_energy

-0.9981493534714043